# Using the models: a very short tutorial

## A first run through

In [ ]:
import models

In [ ]:
# Import time series data

ts_data = models.load_time_series_data(model_name='6_region')
ts_data = ts_data.loc['2017-01-08':'2017-01-15']  # A week in January

In [ ]:
# Create and run model -- warnings about interest rate can be ignored

model = models.SixRegionModel(ts_data=ts_data,
                              run_mode='plan',
                              baseload_integer=False,
                              baseload_ramping=False,
                              allow_unmet=True)
model.run()

In [ ]:
# Get model outputs

model.get_summary_outputs()

## Some more functionality

The models in this repository are subclassed from `Calliope`, an open-source energy modelling framework with some really neat features. See [this link](https://calliope.readthedocs.io/en/stable/index.html) for tutorials and more information. Below, we'll look at two nice ways of analysing results that are inherited from that software. There are a lot more features -- it's recommended to have a look through the tutorial, especially the "analysing a model" section.

The code above gives a high-level overview of relevant model outputs. However, if you wan't all of them, you can export them to csv as follows:

In [ ]:
# Export full set of outputs to CSV -- creates a directory called 'outputs'

model.to_csv('outputs')

In [ ]:
# Make plots of the model outputs (e.g. power or variable cost). This cell does not
# display properly on Github. If you want to see the outputs, open it up yourself
# and run it in your own Jupyter notebook. 

model.plot.timeseries(array='results')